In [1]:
# Importing of necessary libraries
import pandas as pd
import streamlit as st

# Loading of Excel file
excel_file = pd.read_excel('data_source/Twisto_data.xlsx', sheet_name=None)

# Displaying first 3 rows of each sheet (5 tables)
for name, df in excel_file.items():
    print(f"Sheet: {name}")
    display(df.head(3))

Sheet: profile_base


,date_upgraded,source,accepted,card_ordered,active_last_month,active_last_3_months,user
0,2090-10-12 09:55:36.000,now,1,1,1,1,0.000002
1,2018-11-06 12:16:57.000,performance+organic,1,0,0,0,0.000009
2,2019-04-29 16:16:17.357,referral,1,1,1,1,0.000018


Sheet: profile_customer


,gender,date_of_birth,education_type,family_type,family_children,user
0,Male,1978-10-23,Secondary with exam,Partner,0.0,0.010292
1,Female,1977-02-28,University,Single,0.0,0.012073
2,Female,1973-06-30,Secondary,Single,0.0,0.000355


Sheet: profile_extension


,used_extension,user
0,True,0.000002
1,False,0.000009
2,True,0.000018


Sheet: profile_apps


,app_ios,app_android,user
0,0,0,0.000002
1,1,0,0.000009
2,0,0,0.000018


Sheet: profile_transactions


,category,merchant,date_created,id,total_price_vat,transaction_source,user
0,NaN,Twisto Snap,2016-11-02 09:15:17.975,295939,3930.0,Snap,0.000002
1,NaN,Twisto Snap,2016-09-20 09:35:10.649,262629,70.0,Snap,0.000002
2,NaN,Twisto Snap,2016-09-20 09:27:01.354,262622,210.0,Snap,0.000002


In [2]:
# Extracting of individual DataFrames
df_base = excel_file['profile_base']
df_customer = excel_file['profile_customer']
df_extension = excel_file['profile_extension']
df_apps = excel_file['profile_apps']
df_transactions = excel_file['profile_transactions']

In [3]:
# Cleaning up df_base (1st sheet)
# Fixing specific incorrect dates
df_base['date_upgraded'] = df_base['date_upgraded'].replace({
    '2090-10-12 09:55:36.000': '2014-10-12 09:55:36.000',
    '2080-12-23 22:50:47.000': '2015-12-23 22:50:47.000'
})

# Convert to datetime
df_base['date_upgraded'] = pd.to_datetime(df_base['date_upgraded'], errors='coerce')

# Create separate date and time columns (only hours & minutes)
df_base['date_upgr'] = df_base['date_upgraded'].dt.date
df_base['time_upgr'] = df_base['date_upgraded'].dt.strftime('%H:%M')

# Drop the original datetime column
df_base.drop(columns=['date_upgraded'], inplace=True)

# Strip leading/trailing spaces
df_base['source'] = df_base['source'].str.strip()

# Replace specific typos
df_base['source'] = df_base['source'].replace({
    'now saved card': 'now',
    'now': 'now',  # redundant but keeps structure clear
    'referrall': 'referral',
    'affill': 'affiliate',
    'affil': 'affiliate'
})

In [4]:
# Cleaning up df_customer (2nd sheet)
# Clean 'gender' column: Replace 99 with 'Unknown' or a proper missing value
df_customer['gender'] = df_customer['gender'].replace(99, 'Unknown')

# Filling/replacing missing values in other columns with 'Unknown'
df_customer['education_type'] = df_customer['education_type'].fillna('Unknown')
df_customer['family_children'] = df_customer['family_children'].fillna('Unknown')
df_customer['family_children'] = df_customer['family_children'].fillna('Unknown')
df_customer['family_children'] = df_customer['family_children'].replace('-100', 'Unknown')

# 3. Normalize 'education_type': Replace 'Secondary with exam' with 'Secondary'
df_customer['education_type'] = df_customer['education_type'].replace('Secondary with exam', 'Secondary')

# 6. Convert 'family_children' to numeric where possible, errors='coerce' will convert 'Unknown'
df_customer['family_children_num'] = pd.to_numeric(df_customer['family_children'], errors='coerce')

# 7. Create 'family_size' column based on conditions
def family_size_label(x):
    if pd.isna(x):  # this corresponds to 'Unknown' or missing values
        return 'No children'
    elif x == 0:
        return 'No children'
    elif x == 1:
        return 'One children'
    elif x > 1:
        return 'Big family'
    else:
        return 'Unknown'  # fallback for any unexpected values

df_customer['family_size'] = df_customer['family_children_num'].apply(family_size_label)

# Dropping the helper numeric column
df_customer.drop(columns=['family_children_num'], inplace=True)

In [5]:
# Adding the column 'age' in the df_customer
# Ensure date_of_birth is datetime
df_customer['date_of_birth'] = pd.to_datetime(df_customer['date_of_birth'], errors='coerce')
df_base['date_upgr'] = pd.to_datetime(df_base['date_upgr'], errors='coerce')

# Merge only the 'date_upgr' column into df_customer by 'user' key
df_customer = df_customer.merge(
    df_base[['user', 'date_upgr']],
    on='user',
    how='left'
)

# Calculate age at joining and add it as a new column in df_customer
df_customer['age'] = (df_customer['date_upgr'] - df_customer['date_of_birth']).dt.days // 365

# If you don't want to keep the 'date_upgr' column in df_customer, you can drop it
df_customer.drop(columns=['date_upgr'], inplace=True)

# Check result
print(df_customer[['user', 'date_of_birth', 'age']].head())

       user date_of_birth  age
0  0.010292    1978-10-23   39
1  0.012073    1977-02-28   41
2  0.000355    1973-06-30   44
3  0.000998    1978-10-07   39
4  0.009395    1970-08-19   48


In [6]:
# Filling in all empty values in df_transaction (column: category)

df_transactions['category'] = df_transactions['category'].fillna('Unknown')

In [7]:
# Basic checking of number from the file
total_customers = len(df_customer)
male_customers = df_customer[df_customer['gender'].str.lower() == 'male'].shape[0]
female_customers = df_customer[df_customer['gender'].str.lower() == 'female'].shape[0]
unknown_gender = df_customer[df_customer['gender'].str.lower() == 'unknown'].shape[0]
average_age = df_customer['age'].dropna().mean()

total_transactions = len(df_transactions)

ios_users_count = df_apps[df_apps['app_ios'] > 0]['user'].nunique()

android_users_count = df_apps[df_apps['app_android'] > 0]['user'].nunique()


print(f"Total customers: {total_customers}")
print(f"Male customers: {male_customers}")
print(f"Female customers: {female_customers}")
print(f"Unknown gender customers: {unknown_gender}")
print(f"Average age: {average_age:.2f} years")
print(f"Total transactions: {total_transactions}")
print(f"Number of iOS users: {ios_users_count}")
print(f"Number of Android users: {android_users_count}")

Total customers: 1009
Male customers: 526
Female customers: 468
Unknown gender customers: 15
Average age: 31.34 years
Total transactions: 67267
Number of iOS users: 292
Number of Android users: 314


In [8]:
def classify_os(row):
    ios = row['app_ios'] > 0
    android = row['app_android'] > 0
    
    if ios:
        return 'iOS'
    elif android:
        return 'Android'
    elif ios and android:
        return 'Both'  # optional
    else:
        return 'Unknown'

# Create new column 'os_type'
df_apps['os_type'] = df_apps.apply(classify_os, axis=1)

# Count unique users by os_type
os_counts = df_apps.groupby('os_type')['user'].nunique()

print(os_counts)

os_type
Android    281
Unknown    427
iOS        292
Name: user, dtype: int64
